<a href="https://colab.research.google.com/github/anilsinghchawda/ml_new/blob/master/virality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
! pip install requests

In [78]:
! pip install newspaper3k

In [79]:
! pip install beautifulsoup4

In [0]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np

In [0]:
url="https://in.mashable.com/"
r=requests.get(url)

In [0]:
soup=BeautifulSoup(r.content,'html5lib') 
table=soup.findAll('a',attrs={'class':'title'})

In [0]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://in.mashable.com/'+row['href'])

In [0]:
import nltk

In [85]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

In [87]:
dataset=pd.DataFrame(df)
dataset.head()

,Title,Text,Summary,Keywords
0,We've Rounded Up The 20 Best Tweets From This ...,"Another week of quarantine, another week of tw...","Another week of quarantine, another week of tw...","[tweets, doll, 26, 27, 29, best, weve, week, 2..."
1,"Reddit VP: 'Ya, We F*cked Up.'",Reddit botched the rollout of a new feature th...,Reddit botched the rollout of a new feature th...,"[rrape, fcked, communities, moderators, vp, re..."
2,The 16 Most Exciting Movies And Shows Coming S...,"Summer 2020 may seem a little different, but t...",Whether you're cueing up a weekend binge or a ...,"[coming, premieres, exciting, season, series, ..."
3,Eta Aquarids Meteor Shower Will Light Up An Al...,"Not too long ago, the Lyrids meteor shower lit...","The Eta Aquarids, one of the two annual meteor...","[eta, trail, night, shower, showers, comet, li..."
4,Asteroid Shaped Like A Face Mask Passes Earth....,If you believe in signs from the universe or G...,If you believe in signs from the universe or G...,"[universes, psa, face, or2, threat, passes, un..."


In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [0]:
FILEPATH="OnlineNewsPopularity.csv"

In [97]:
data1=pd.read_csv(FILEPATH)
data1.head(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593.0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711.0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500.0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200.0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505.0


In [0]:
clean_cols={x: x.lower().strip() for x in list(data1)}
data1=data1.rename(index=str,columns=clean_cols)

In [0]:
train_set, test_set = train_test_split(data1, test_size=0.4, random_state=42)

x_train = train_set.drop(['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess'], axis=1)
y_test = test_set['shares']

In [104]:
np.all(np.isfinite(x_train))

False

In [105]:
np.all(np.isfinite(y_train))

False

In [0]:
x_train = x_train[1:3000:]

In [0]:
y_train = y_train[1:3000:]

In [143]:
print(y_train)

1963    3800.0
6277    1300.0
3651     906.0
6647    3300.0
3199    2300.0
         ...  
219      762.0
8136     639.0
1288     686.0
3163    1900.0
582     1100.0
Name: shares, Length: 399, dtype: float64


In [144]:
print(x_train)

      n_tokens_title  ...  abs_title_sentiment_polarity
1963            10.0  ...                          0.00
6277            10.0  ...                          0.00
3651            11.0  ...                          0.00
6647             8.0  ...                          0.25
3199            11.0  ...                          0.50
...              ...  ...                           ...
219             13.0  ...                          0.00
8136            12.0  ...                          0.00
1288             9.0  ...                          0.15
3163             9.0  ...                          0.00
582             10.0  ...                          0.25

[399 rows x 45 columns]


In [151]:
clf=RandomForestRegressor(random_state=12)
clf.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=12, verbose=0, warm_start=False)

In [0]:
result=pd.DataFrame(clf.predict(x_train),list(y_train))


In [153]:
result.reset_index(level=0, inplace=True)
result_df = result.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
result_df.head()

,Actual shares,Predicted shares
0,1300.0,1602.99
1,906.0,4529.32
2,3300.0,2727.59
3,2300.0,2172.78
4,1200.0,1590.08


In [154]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print(len(documents))
print(documents[:5])

99999
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [0]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

stemmer=PorterStemmer()

In [0]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [160]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [161]:
doc_sample = documents[documents['index'] ==2].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['a', 'g', 'calls', 'for', 'infrastructure', 'protection', 'summit']


 tokenized and lemmatized document: 
['call', 'infrastructur', 'protect', 'summit']


In [162]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [163]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print(dictionary)
count=0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 15:
        break

Dictionary(16970 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
11 aust
12 rise
13 staff
14 strike
15 affect


In [164]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[2]

[(7, 1), (8, 1), (9, 1), (10, 1)]

In [165]:
bow_doc_2 = bow_corpus[2]
for i in range(len(bow_doc_2)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                               dictionary[bow_doc_2[i][0]], 
bow_doc_2[i][1]))

Word 7 ("call") appears 1 time.
Word 8 ("infrastructur") appears 1 time.
Word 9 ("protect") appears 1 time.
Word 10 ("summit") appears 1 time.


In [166]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"polic" + 0.023*"charg" + 0.017*"court" + 0.012*"face" + 0.011*"concern" + 0.011*"murder" + 0.010*"drug" + 0.009*"death" + 0.008*"appeal" + 0.008*"kill"
Topic: 1 
Words: 0.025*"iraq" + 0.014*"polic" + 0.014*"water" + 0.011*"miss" + 0.009*"iraqi" + 0.008*"kill" + 0.008*"attack" + 0.007*"bush" + 0.007*"free" + 0.007*"continu"
Topic: 2 
Words: 0.016*"say" + 0.015*"boost" + 0.012*"lead" + 0.010*"plan" + 0.008*"win" + 0.008*"elect" + 0.008*"labor" + 0.008*"council" + 0.007*"tell" + 0.007*"fight"
Topic: 3 
Words: 0.018*"urg" + 0.013*"govt" + 0.011*"fund" + 0.009*"prison" + 0.009*"help" + 0.008*"high" + 0.007*"abus" + 0.007*"australian" + 0.007*"consid" + 0.007*"olymp"
Topic: 4 
Words: 0.022*"plan" + 0.010*"govt" + 0.010*"council" + 0.010*"support" + 0.010*"strike" + 0.009*"protest" + 0.008*"hospit" + 0.008*"target" + 0.008*"polic" + 0.007*"defend"


In [167]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.35103660821914673	 
Topic: 0.016*"say" + 0.015*"boost" + 0.012*"lead" + 0.010*"plan" + 0.008*"win"

Score: 0.3022243082523346	 
Topic: 0.025*"iraq" + 0.014*"polic" + 0.014*"water" + 0.011*"miss" + 0.009*"iraqi"

Score: 0.2646525204181671	 
Topic: 0.018*"urg" + 0.013*"govt" + 0.011*"fund" + 0.009*"prison" + 0.009*"help"

Score: 0.04116009920835495	 
Topic: 0.022*"plan" + 0.010*"govt" + 0.010*"council" + 0.010*"support" + 0.010*"strike"

Score: 0.04092644155025482	 
Topic: 0.025*"polic" + 0.023*"charg" + 0.017*"court" + 0.012*"face" + 0.011*"concern"


In [0]:
def get_LDA(text):
    unseen_document = text
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    result=[]
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        #print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
        result.append(score)
    return result

In [170]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))

In [0]:
def for_unique_tokens_rate(words):
    words=tokenize(words)
    no_order = list(set(words))
    unique_tokens_rate=len(no_order)/len(words)
    return unique_tokens_rate

In [0]:
def for_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    nonstop_words_rate=len(filtered_sentence)/len(words)
    return nonstop_words_rate

In [0]:
def for_unique_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    no_order = list(set(filtered_sentence))
    unique_nonstop_words_rate=len(no_order)/len(words)
    return unique_nonstop_words_rate

In [0]:
def avg_token_length(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

In [178]:
! pip install datefinder

In [0]:
from textblob import TextBlob
import datefinder
import datetime  
from datetime import date

In [0]:
def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article_text)))>0:
        date=str(list(datefinder.find_dates(article_text))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday"

In [0]:

def tokenize(text):
    text=text
    return word_tokenize(text)

In [0]:
def non_neutral_tokens(text):
    processed_docs = preprocess(text)
    speech=nltk.pos_tag(processed_docs)
    tags=["NN","NNP","NNS","JJ","JJR","JJS","RB","RBR","RBS","UH"]
    speech_list=[i[0] for i in speech if i[1] in tags]
    return len(speech_list)

In [0]:
positive_words=[]
negative_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            positive_words.append(i)
        if polarity<0:
            negative_words.append(i)
    return positive_words,negative_words

In [0]:
def rates(words):
    words=polar(words)
    positive=words[0]
    negative=words[1]
    global_rate_positive_words=(len(positive)/len(words))/100
    global_rate_negative_words=(len(negative)/len(words))/100
    polar_positive=[]
    polar_negative=[]
    for i in positive:
        analysis=TextBlob(i)
        polar_positive.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity
    for j in negative:
        analysis2=TextBlob(j)
        polar_negative.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity
    min_positive_polarity=min(polar_positive)
    max_positive_polarity=max(polar_positive)
    min_negative_polarity=min(polar_negative)
    max_negative_polarity=max(polar_negative)
    avg_positive_polarity=np.average(polar_positive)
    avg_negative_polarity=np.average(polar_negative)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [186]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
df1=[]
for i in news:
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=for_unique_tokens_rate(article.text)
    pred_info['n_non_stop_words']=for_nonstop_words_rate(article.text)
    pred_info['n_non_stop_unique_tokens']=for_unique_nonstop_words_rate(article.text)
    pred_info['num_hrefs']=article.html.count("https://in.mashable.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token_length(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    if "life-style" in article.url:
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(i)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(i)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(i)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(i)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(i)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(i)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(i)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
        
    pred_info['LDA_00']=get_LDA(article.text)[0]
    pred_info['LDA_01']=get_LDA(article.text)[1]
    pred_info['LDA_02']=get_LDA(article.text)[2]
    pred_info['LDA_03']=get_LDA(article.text)[3]
    pred_info['LDA_04']=get_LDA(article.text)[4]
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]
    pred_info['rate_positive_words']=len(polar(article.text)[0])/non_neutral_tokens(article.text)
    pred_info['rate_negative_words']=len(polar(article.text)[1])/non_neutral_tokens(article.text)
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    pred_info['abs_title_subjectivity']=abs(title_analysis.sentiment.subjectivity-0.5)
    pred_info['abs_title_sentiment_polarity']=abs(title_analysis.sentiment.polarity)
    df1.append(pred_info)

In [188]:
pred_df=pd.DataFrame(df1)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()


,text,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,rate_positive_words,rate_negative_words,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,"Another week of quarantine, another week of tw...",11,932,0.497854,0.739270,0.417382,38,8,0,3.769313,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0.281248,0.227969,0.209018,0.154726,0.127039,0.492492,0.244037,1.800,0.495,0.486455,0.160000,1.0,-0.330247,-1.0,-0.05,2.382353,0.628676,0.30,1.0,0.20,1.0
1,Reddit botched the rollout of a new feature th...,10,804,0.461443,0.665423,0.371891,36,8,0,4.444030,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0.272944,0.216206,0.188336,0.175775,0.146732,0.550174,0.072970,3.575,0.995,0.464750,0.033333,1.0,-0.347650,-1.0,-0.05,3.593284,1.160448,0.00,0.0,0.50,0.0
2,"Summer 2020 may seem a little different, but t...",11,1559,0.454779,0.751764,0.407312,36,9,0,4.310455,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0.346000,0.197483,0.176961,0.175350,0.104206,0.486069,0.163423,5.225,1.805,0.434118,0.033333,1.0,-0.338923,-1.0,-0.05,2.249226,0.868421,0.65,0.4,0.15,0.4
3,"Not too long ago, the Lyrids meteor shower lit...",11,748,0.393048,0.664439,0.323529,36,8,0,4.096257,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0.419351,0.198940,0.173253,0.138012,0.070447,0.471359,0.099125,7.600,2.935,0.440135,0.033333,1.0,-0.319802,-1.0,-0.05,6.488189,2.720472,0.70,0.4,0.20,0.4
4,If you believe in signs from the universe or G...,15,425,0.543529,0.672941,0.432941,36,9,0,4.188235,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0.291562,0.230342,0.181477,0.169424,0.127196,0.409006,0.054167,8.360,3.515,0.437543,0.033333,1.0,-0.308315,-1.0,-0.05,14.466667,6.258333,0.00,0.0,0.50,0.0


In [189]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,"Another week of quarantine, another week of tw...",8103.14
1,Reddit botched the rollout of a new feature th...,7914.83
2,"Summer 2020 may seem a little different, but t...",9060.61
3,"Not too long ago, the Lyrids meteor shower lit...",7999.69
4,If you believe in signs from the universe or G...,7971.53
5,Qualcomm recently reported its second quarter ...,7484.97
6,Palaeontologists have discovered a cat-sized m...,7061.17
7,Twitter recently announced that it will allow ...,8035.32
8,These are dark times. We as a species have bee...,8043.20
9,Mobile streaming service Quibi is less than a ...,7688.23
